In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd
import numpy as np
import requests 
import ffmpeg
import ffprobe3
import speech_recognition as sr
from pydub import AudioSegment
import sys

/home/dataiku/dss/code-envs/python/podcast_reader_ffmpeg/lib/python2.7/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [ ]:
#downloads_dir = r"episodes_wavs"

In [3]:
r = sr.Recognizer()

In [12]:
# Read recipe inputs
episodes_sample = dataiku.Dataset("episodes_sample")
episodes_sample_df = episodes_sample.get_dataframe()

In [6]:
episode_wavs_folder = dataiku.Folder("episoide_wavs")

In [7]:
folder_path = episode_wavs_folder.get_path()

In [10]:
folder_path

u'/home/dataiku/dss/managed_datasets/PODCAST_READER/7PhFmxxL'

In [13]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

episodes_text_df = episodes_sample_df # For this sample code, simply copy input to output
episodes_text_df["episode_text"] = ""
list_of_text = []

already_read = 0
for row in episodes_sample.iter_rows():
    print(row)
    # row is a dict-like object
    print(str(already_read+1) +": " + row['audio_url'])
    
    url = row['audio_url']
    file = requests.get(url)
    open(folder_path + "/audio_file.mp3","wb").write(file.content)
    sound = AudioSegment.from_mp3(folder_path + "/audio_file.mp3")
    sound.export(downloads_dir + "audio_file.wav", format="wav")
    
    total_time = round(sound.duration_seconds)
    offset = 0
    duration = 60
    subsegments = int(round((total_time-offset)/duration))
    
    s = ""
    with sr.AudioFile(downloads_dir + "audio_file.wav") as source:
        for subsegment in range(1,subsegments):
            print("Subsegment " + str(subsegment) + " out of " + str(subsegments))
        
            audio = r.record(source,offset=offset+(subsegment-1)*duration,duration=duration)  # read the entire audio file                  
            try:
                s= s + " " + r.recognize_google(audio)
            except:
                s = s + " NA"
    print(s)      
    list_of_text.append(s)

    already_read += 1
    if already_read == 5:
        break

episodes_text_df["episode_text"] = pd.Series(list_of_text)        

# Write recipe outputs
episodes_text = dataiku.Dataset("episodes_text")
episodes_text.write_with_schema(episodes_text_df)

{u'category': u'Voyager', u'show_id': u'580ca3d7-1d85-3ef3-a1b4-c6f4edf9ff65', u'subtitle': u'Top Five Out-of-Character Moments. From small gripes to glaring errors, Char and Tristan run through their top five out-of-character moments on Voyager in this episode of To The Journey.', u'description': u"<h3>Top Five Out-of-Character Moments.&nbsp;</h3><p>Whether it's due to a temporal rift or an alien virus, one of the most common ways that we as a Star Trek audience know that something isn't right is by making a character say or do something they normal", u'title': u'To The Journey 138: George Clooney Found His Janeway', u'audio_mime_type': u'audio/mpeg', u'author': u'Trek.fm', u'explicit': u'0', u'audio_file_size': u'32657154', u'summary': u"Top Five Out-of-Character Moments.   Whether it's due to a temporal rift or an alien virus, one of the most common ways that we as a Star Trek audience know that something isn't right is by making a character say or do something they normally would n

IOError: [Errno 13] Permission denied: u'/home/dataiku/dss/managed_datasets/PODCAST_READER/7PhFmxxL/audio_file.mp3'